### Hashing 

A hash function is a function that takes an inpute and deterministically converts it to an integer that is less than a fized size set by the programmer. Inputs are called **keys** and the same input will always be converted to the same integer.

**Check if the sentence is Pangram**
- A pangram is a sentence where every letter of the English alphabet appears at least once.

In [2]:
#Q: Given a string sentence containing only lowercase English letters, return true if sentence is a pangram or false otherwise
def is_pangram(sentence):
    seen = set()

    for i in sentence:
        seen.add(i)
        if len(seen) == 26:
            return True 
        
    return len(seen) == 26 

if __name__ == '__main__':
    sentence = "thequickbrownfoxjumpsoverthelazydog"
    print(is_pangram(sentence))

True


### Create A HashTable Or Hash Function/ Hash Map: Separate Chaining

Python implementation of a simple hash table (also known as a hash map) using **separate chaning** to handle **collision** including basic operators or methods such as **set**, **get** and **delete** 

In [1]:
class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize the table with a fixed size.
        Each slot in the table contains a list to handle collision via chaining.
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)]

    def _hash(self, key):
        """
        Generate a hash for the given key.
        Using the python built-in hash function 
        """
        return hash(key) % self.size 
    
    def set(self, key, value):
        """ 
        Insert a key-value pair into the hash table 
        If the key already exist, update the value
        """
        index =  self._hash(key)
        #check if the key already exist and update the value
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value
                print(f"Updated key '{key}' with value '{value}'.")
                return 
        
        #If the key doesn't exist, append new key-value pair
        self.table[index].append([key, value])
        print(f"Inserted key '{key}' with value '{value}'.")

    def get(self, key):
        """
        Retrieve the value associated with the given key 
        Returns None if the key is not Found 
        """
        index = self._hash(key)
        for pair in self.table[index]:
            if pair[0] == key:
                print(f"Found key '{key}' with value '{pair[1]}'.")
                return pair[1]
        
        print(f"Key '{key}' not Found")
        return None
    
    def delete(self, key):
        """
        Remove the key-value pair associated with the given key from the hash table  
        """
        index = self._hash(key)
        for i, pair in enumerate(self.table[index]):
            if pair[0] == key:
                del self.table[index][i]
                print(f"Deleted key '{key}'.")
                return 
        print(f"Key '{key }' not found. Nothing to delete.")

    def __str__(self):
        """ 
        For debugging: Return a string representation of the hash table.
        """
        table_str = ""
        for i, bucket in enumerate(self.table):
            if bucket:
                table_str += f"Bucket {i}: " + ", ".join([f"{k}: {v}" for k, v in bucket]) + "\n"
        return table_str if table_str else "hash Table is empty."
    

if __name__ == '__main__':
    ht = HashTable(size=10)

    #Insert key-value pairs
    ht.set("apple", 1)
    ht.set("banana", 2)
    ht.set("orange", 3)
    ht.set("apple", 10)  # Update existing key

    # Retrieve values
    print(ht.get("apple"))   # Output: 10
    print(ht.get("banana"))  # Output: 2
    print(ht.get("grape"))   # Output: None

    # Delete a key
    ht.delete("banana")
    print(ht.get("banana"))  # Output: None

    # Print the current state of the hash table
    print(ht)



Inserted key 'apple' with value '1'.
Inserted key 'banana' with value '2'.
Inserted key 'orange' with value '3'.
Updated key 'apple' with value '10'.
Found key 'apple' with value '10'.
10
Found key 'banana' with value '2'.
2
Key 'grape' not Found
None
Deleted key 'banana'.
Key 'banana' not Found
None
Bucket 1: apple: 10
Bucket 6: orange: 3



**Explanation**:
